# Part 3: Paper Review Agent with LangGraph

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. **Agent** 개념 이해하기
2. **LangGraph**로 단계별 워크플로우 구현
3. 논문 리뷰 Agent 만들기 (7차원 평가)
4. 각 단계별 출력 확인하기

---
## Part 1: Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경: 프로젝트 루트로 이동
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
if IN_COLAB:
    print("📦 패키지 설치 중...")
    %pip install langgraph langchain-openai langchain-google-genai python-dotenv -q
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")

In [ ]:
# Cell 3: API Key 로딩 및 LLM 선택
import os

# API Key 로딩
if IN_COLAB:
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    except:
        OPENAI_API_KEY = None
        GEMINI_API_KEY = None
else:
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# LLM 선택 (OpenAI 우선, Gemini 대안)
LLM_PROVIDER = None

if OPENAI_API_KEY and not OPENAI_API_KEY.startswith('your_'):
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    LLM_PROVIDER = 'openai'
    print("✅ OpenAI API 사용 (기본)")
elif GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    LLM_PROVIDER = 'gemini'
    print("✅ Gemini API 사용 (OpenAI 없음)")
else:
    print("❌ API Key가 필요합니다!")
    print("   .env 파일에 OPENAI_API_KEY 또는 GEMINI_API_KEY를 설정하세요.")

In [ ]:
# Cell 4: LLM 초기화
if LLM_PROVIDER == 'openai':
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    print(f"🤖 LLM: OpenAI gpt-4o-mini")
    
elif LLM_PROVIDER == 'gemini':
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)
    print(f"🤖 LLM: Google Gemini 1.5 Flash")
    
else:
    llm = None
    print("⚠️ LLM을 초기화할 수 없습니다.")

In [ ]:
# Cell 5: 샘플 논문 로딩
import os

# 샘플 논문 경로
SAMPLE_PATH = os.path.join(WORKSHOP_DIR, "input", "sample_manuscript.md")

if os.path.exists(SAMPLE_PATH):
    with open(SAMPLE_PATH, 'r', encoding='utf-8') as f:
        SAMPLE_PAPER = f.read()
    print(f"✅ 샘플 논문 로딩: {SAMPLE_PATH}")
else:
    # 파일이 없으면 기본 샘플 사용
    SAMPLE_PAPER = """
TITLE: The Effects of Mindfulness-Based Cognitive Therapy on Depression Relapse Prevention

ABSTRACT:
This randomized controlled trial evaluated MBCT compared to treatment as usual (TAU) 
in preventing depression relapse over 12 months. 256 participants with recurrent MDD 
in remission were randomized. Results showed MBCT+TAU had significantly lower relapse 
rates (31%) compared to TAU (48%) at 12 months (HR=0.58, p=.008).

METHODS:
Participants were randomized to MBCT+TAU (n=128) or TAU alone (n=128). Primary outcome 
was time to relapse. Secondary outcomes included mindfulness skills, rumination, and 
self-compassion.

RESULTS:
Mediation analyses revealed that increases in mindfulness skills and decreases in 
rumination partially mediated treatment effects.

CONCLUSIONS:
MBCT effectively prevents depression relapse with mindfulness skills and reduced 
rumination serving as key mechanisms.
"""
    print("⚠️ 샘플 파일 없음 - 기본 샘플 사용")

# 논문 정보 출력
lines = SAMPLE_PAPER.strip().split('\n')
title = lines[0].replace('#', '').strip() if lines else "Unknown"
print(f"\n📄 샘플 논문 준비 완료")
print(f"   제목: {title[:60]}...")
print(f"   길이: {len(SAMPLE_PAPER)} 문자")

---
## Part 2: Agent 개념 소개

### Agent란?

**Agent** = LLM + 도구 + 워크플로우

단순 프롬프트와의 차이:

| | 단순 프롬프트 | Agent |
|--|-------------|-------|
| 구조 | 한 번에 모든 것 | 단계별 처리 |
| 중간 결과 | 볼 수 없음 | 각 단계 확인 가능 |
| 에러 처리 | 처음부터 다시 | 실패한 단계만 재시도 |
| 복잡한 작업 | 품질 불안정 | 안정적 |

### LangGraph 기본 구조

```
[시작] → [Node 1] → [Node 2] → [Node 3] → [종료]
           ↓           ↓           ↓
        Output 1    Output 2    Output 3
```

각 **Node**는 하나의 작업을 수행하고, 결과를 **State**에 저장합니다.

In [ ]:
# Cell 6: LangGraph 기본 구조 정의
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END

# State: Agent가 공유하는 데이터
class ReviewState(TypedDict):
    paper: str              # 입력 논문
    summary: str            # 요약
    strengths: str          # 강점
    weaknesses: str         # 약점
    scores: dict            # 7차원 점수
    final_review: str       # 최종 리뷰

print("✅ State 정의 완료")
print("")
print("ReviewState 구조:")
print("  - paper: 입력 논문")
print("  - summary: 논문 요약")
print("  - strengths: 강점 분석")
print("  - weaknesses: 약점 분석")
print("  - scores: 7차원 평가 점수")
print("  - final_review: 최종 리뷰")

---
## Part 3: 단계별 Node 구현

이제 각 Node를 하나씩 구현하고 실행해봅니다.

### Node 구조
```
[논문] → [요약] → [강점] → [약점] → [7차원 평가] → [최종 리뷰]
```

In [ ]:
# Cell 7: Node 1 - 논문 요약
from langchain_core.prompts import ChatPromptTemplate

def summarize_paper(state: ReviewState) -> dict:
    """논문을 2-3문장으로 요약"""
    prompt = ChatPromptTemplate.from_template("""
You are an expert academic reviewer. Summarize the following paper in 2-3 sentences.
Focus on: research question, methodology, and main findings.

Paper:
{paper}

Summary (2-3 sentences):
""")
    
    chain = prompt | llm
    result = chain.invoke({"paper": state["paper"]})
    return {"summary": result.content}

# 테스트 실행
print("🔄 Node 1: 논문 요약 실행 중...")
print("="*60)

test_state = {"paper": SAMPLE_PAPER}
result = summarize_paper(test_state)

print("📝 [Output] 요약:")
print(result["summary"])
print("="*60)

In [ ]:
# Cell 8: Node 2 - 강점 분석
def analyze_strengths(state: ReviewState) -> dict:
    """논문의 강점 3-5개 분석"""
    prompt = ChatPromptTemplate.from_template("""
You are an expert academic reviewer. Identify 3-5 key strengths of this paper.
Consider: methodology, contribution, clarity, and significance.

Paper:
{paper}

Summary of the paper:
{summary}

List the strengths (3-5 bullet points):
""")
    
    chain = prompt | llm
    result = chain.invoke({
        "paper": state["paper"],
        "summary": state.get("summary", "")
    })
    return {"strengths": result.content}

# 테스트 실행
print("🔄 Node 2: 강점 분석 실행 중...")
print("="*60)

test_state["summary"] = result["summary"]  # 이전 결과 연결
result2 = analyze_strengths(test_state)

print("💪 [Output] 강점:")
print(result2["strengths"])
print("="*60)

In [ ]:
# Cell 9: Node 3 - 약점 분석
def analyze_weaknesses(state: ReviewState) -> dict:
    """논문의 약점 3-5개 분석"""
    prompt = ChatPromptTemplate.from_template("""
You are an expert academic reviewer. Identify 3-5 key weaknesses or areas for improvement.
Be constructive and specific. Consider: methodology limitations, missing analyses, clarity issues.

Paper:
{paper}

Strengths identified:
{strengths}

List the weaknesses (3-5 bullet points, be constructive):
""")
    
    chain = prompt | llm
    result = chain.invoke({
        "paper": state["paper"],
        "strengths": state.get("strengths", "")
    })
    return {"weaknesses": result.content}

# 테스트 실행
print("🔄 Node 3: 약점 분석 실행 중...")
print("="*60)

test_state["strengths"] = result2["strengths"]  # 이전 결과 연결
result3 = analyze_weaknesses(test_state)

print("⚠️ [Output] 약점:")
print(result3["weaknesses"])
print("="*60)

In [ ]:
# Cell 10: Node 4 - 7차원 평가
import json
import re

def evaluate_dimensions(state: ReviewState) -> dict:
    """7차원으로 논문 평가 (Stanford Agentic Reviewer 방식)"""
    prompt = ChatPromptTemplate.from_template("""
You are an expert academic reviewer. Rate this paper on 7 dimensions (1-10 scale).

Paper:
{paper}

Strengths: {strengths}
Weaknesses: {weaknesses}

Rate each dimension (1-10) and provide a brief justification:

1. Originality: How novel is the research question or approach?
2. Significance: How important is the contribution to the field?
3. Soundness: Is the methodology rigorous and appropriate?
4. Clarity: Is the paper well-written and easy to follow?
5. Reproducibility: Can the study be replicated?
6. Ethics: Are ethical considerations addressed?
7. Overall: Overall quality of the paper.

Respond in this exact JSON format:
{{
    "originality": {{"score": X, "reason": "..."}},
    "significance": {{"score": X, "reason": "..."}},
    "soundness": {{"score": X, "reason": "..."}},
    "clarity": {{"score": X, "reason": "..."}},
    "reproducibility": {{"score": X, "reason": "..."}},
    "ethics": {{"score": X, "reason": "..."}},
    "overall": {{"score": X, "reason": "..."}}
}}
""")
    
    chain = prompt | llm
    result = chain.invoke({
        "paper": state["paper"],
        "strengths": state.get("strengths", ""),
        "weaknesses": state.get("weaknesses", "")
    })
    
    # JSON 파싱 시도
    try:
        # JSON 블록 추출
        json_match = re.search(r'\{[\s\S]*\}', result.content)
        if json_match:
            scores = json.loads(json_match.group())
        else:
            scores = {"error": "JSON 파싱 실패", "raw": result.content}
    except:
        scores = {"error": "JSON 파싱 실패", "raw": result.content}
    
    return {"scores": scores}

# 테스트 실행
print("🔄 Node 4: 7차원 평가 실행 중...")
print("="*60)

test_state["weaknesses"] = result3["weaknesses"]  # 이전 결과 연결
result4 = evaluate_dimensions(test_state)

print("📊 [Output] 7차원 평가:")
if "error" not in result4["scores"]:
    for dim, data in result4["scores"].items():
        print(f"  {dim}: {data['score']}/10 - {data['reason'][:50]}...")
else:
    print(result4["scores"])
print("="*60)

In [ ]:
# Cell 11: Node 5 - 최종 리뷰 생성
def generate_final_review(state: ReviewState) -> dict:
    """모든 분석을 종합하여 최종 리뷰 생성"""
    
    # 점수 요약
    scores = state.get("scores", {})
    if "error" not in scores:
        score_summary = ", ".join([f"{k}: {v['score']}/10" for k, v in scores.items()])
    else:
        score_summary = "점수 없음"
    
    prompt = ChatPromptTemplate.from_template("""
You are an expert academic reviewer. Write a comprehensive final review.

Paper Summary:
{summary}

Strengths:
{strengths}

Weaknesses:
{weaknesses}

Scores: {score_summary}

Write a final review that includes:
1. Brief summary (1-2 sentences)
2. Major strengths (2-3 points)
3. Major weaknesses (2-3 points)
4. Questions for authors (2-3 questions)
5. Recommendation (Accept / Minor Revision / Major Revision / Reject)

Final Review:
""")
    
    chain = prompt | llm
    result = chain.invoke({
        "summary": state.get("summary", ""),
        "strengths": state.get("strengths", ""),
        "weaknesses": state.get("weaknesses", ""),
        "score_summary": score_summary
    })
    return {"final_review": result.content}

# 테스트 실행
print("🔄 Node 5: 최종 리뷰 생성 중...")
print("="*60)

test_state["scores"] = result4["scores"]  # 이전 결과 연결
result5 = generate_final_review(test_state)

print("📋 [Output] 최종 리뷰:")
print(result5["final_review"])
print("="*60)

---
## Part 4: Agent 구성 및 결과 저장

이제 모든 Node를 LangGraph로 연결하고, Part 3에서 실행한 결과를 저장합니다.

In [ ]:
# Cell 12: LangGraph로 전체 Agent 구성
from langgraph.graph import StateGraph, END

# 그래프 생성
workflow = StateGraph(ReviewState)

# 노드 추가
workflow.add_node("summarize", summarize_paper)
workflow.add_node("strengths", analyze_strengths)
workflow.add_node("weaknesses", analyze_weaknesses)
workflow.add_node("evaluate", evaluate_dimensions)
workflow.add_node("final_review", generate_final_review)

# 엣지 연결 (순서 정의)
workflow.set_entry_point("summarize")
workflow.add_edge("summarize", "strengths")
workflow.add_edge("strengths", "weaknesses")
workflow.add_edge("weaknesses", "evaluate")
workflow.add_edge("evaluate", "final_review")
workflow.add_edge("final_review", END)

# 컴파일
app = workflow.compile()

print("✅ LangGraph Agent 구성 완료!")
print("")
print("워크플로우:")
print("  [시작] → [요약] → [강점] → [약점] → [7차원 평가] → [최종 리뷰] → [종료]")

In [ ]:
# Cell 13: Part 3 결과 저장 (다음 노트북 비교용)
# 위에서 이미 단계별로 실행했으므로 그 결과를 저장합니다.
import json
import os

# Part 3에서 실행한 결과 취합
review_result = {
    "method": "basic_langgraph_agent",
    "llm_provider": LLM_PROVIDER,
    "paper_title": SAMPLE_PAPER.split('TITLE:')[1].split('ABSTRACT:')[0].strip()[:100],
    "summary": result.get("summary", ""),
    "strengths": result2.get("strengths", ""),
    "weaknesses": result3.get("weaknesses", ""),
    "scores": result4.get("scores", {}),
    "final_review": result5.get("final_review", "")
}

# 저장
output_dir = os.path.join(WORKSHOP_DIR, "outputs", "reviews")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "review_basic.json")

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(review_result, f, ensure_ascii=False, indent=2)

print("💾 결과 저장 완료!")
print(f"   경로: {output_path}")
print("")
print("📊 저장된 내용:")
print(f"   - 요약: {len(review_result['summary'])} 문자")
print(f"   - 강점: {len(review_result['strengths'])} 문자")
print(f"   - 약점: {len(review_result['weaknesses'])} 문자")
print(f"   - 7차원 점수: {len(review_result['scores'])} 항목")
print(f"   - 최종 리뷰: {len(review_result['final_review'])} 문자")
print("")
print("👉 다음 노트북(Few-shot Learning)에서 이 결과와 비교합니다.")

---
## Part 5: DIY - 본인 논문으로 실행

본인의 논문/원고 파일을 입력하여 리뷰를 받아보세요.

**지원 형식:**
- `.txt` - 텍스트 파일
- `.pdf` - PDF 파일 (PyPDF2 필요)
- `.docx` - Word 문서 (python-docx 필요)
- 또는 직접 텍스트 붙여넣기

In [ ]:
# DIY Cell 1: 본인 논문 파일 경로 입력
# PDF, TXT, MD, 또는 Word 문서 경로를 입력하세요

# 방법 1: 파일 경로 입력 (권장)
# 예시: os.path.join(WORKSHOP_DIR, "input", "my_paper.pdf")
MY_PAPER_PATH = ""  # <- 파일 경로 입력

# 💡 샘플을 다시 사용하려면:
# MY_PAPER_PATH = os.path.join(WORKSHOP_DIR, "input", "sample_manuscript.md")

# 방법 2: 직접 텍스트 입력 (파일이 없는 경우)
MY_PAPER_TEXT = """
TITLE: [논문 제목]

ABSTRACT:
[초록을 여기에 붙여넣으세요]

METHODS:
[방법론 요약]

RESULTS:
[주요 결과]

CONCLUSIONS:
[결론]
"""

# 논문 로딩
MY_PAPER = None

if MY_PAPER_PATH:
    if os.path.exists(MY_PAPER_PATH):
        ext = os.path.splitext(MY_PAPER_PATH)[1].lower()
        
        if ext in ['.txt', '.md']:
            with open(MY_PAPER_PATH, 'r', encoding='utf-8') as f:
                MY_PAPER = f.read()
            print(f"✅ 텍스트 파일 로딩: {MY_PAPER_PATH}")
            
        elif ext == '.pdf':
            try:
                import PyPDF2
                with open(MY_PAPER_PATH, 'rb') as f:
                    reader = PyPDF2.PdfReader(f)
                    MY_PAPER = "\n".join([page.extract_text() for page in reader.pages])
                print(f"✅ PDF 파일 로딩: {MY_PAPER_PATH}")
            except ImportError:
                print("⚠️ PyPDF2가 필요합니다: pip install PyPDF2")
            except Exception as e:
                print(f"❌ PDF 로딩 실패: {e}")
                
        elif ext in ['.docx', '.doc']:
            try:
                import docx
                doc = docx.Document(MY_PAPER_PATH)
                MY_PAPER = "\n".join([para.text for para in doc.paragraphs])
                print(f"✅ Word 파일 로딩: {MY_PAPER_PATH}")
            except ImportError:
                print("⚠️ python-docx가 필요합니다: pip install python-docx")
            except Exception as e:
                print(f"❌ Word 로딩 실패: {e}")
        else:
            print(f"⚠️ 지원하지 않는 파일 형식: {ext}")
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {MY_PAPER_PATH}")

# 파일이 없으면 직접 입력한 텍스트 사용
if MY_PAPER is None and len(MY_PAPER_TEXT) > 200:
    MY_PAPER = MY_PAPER_TEXT
    print("✅ 직접 입력한 텍스트 사용")

# 결과 확인
if MY_PAPER and len(MY_PAPER) > 200:
    print(f"\n📄 논문 준비 완료!")
    print(f"   길이: {len(MY_PAPER)} 문자")
    print(f"   미리보기: {MY_PAPER[:150].replace(chr(10), ' ')}...")
else:
    print("\n⚠️ 논문을 입력해주세요!")
    print("   1. MY_PAPER_PATH에 파일 경로 입력")
    print("   2. 또는 MY_PAPER_TEXT에 직접 붙여넣기")
    print(f"\n   📁 input 폴더: {os.path.join(WORKSHOP_DIR, 'input')}")

In [ ]:
# DIY Cell 2: 본인 논문 리뷰 실행
if MY_PAPER and len(MY_PAPER) > 200:
    print("🚀 본인 논문 리뷰 시작!")
    print("="*60)
    
    # Agent 실행
    my_state = {
        "paper": MY_PAPER,
        "summary": "",
        "strengths": "",
        "weaknesses": "",
        "scores": {},
        "final_review": ""
    }
    
    my_result = app.invoke(my_state)
    
    # 결과 출력
    print("\n📋 최종 리뷰:")
    print("="*60)
    print(my_result["final_review"])
    print("="*60)
    
    # 결과 저장
    my_review_result = {
        "method": "basic_langgraph_agent",
        "llm_provider": LLM_PROVIDER,
        "source": MY_PAPER_PATH if MY_PAPER_PATH else "direct_input",
        "summary": my_result.get("summary", ""),
        "strengths": my_result.get("strengths", ""),
        "weaknesses": my_result.get("weaknesses", ""),
        "scores": my_result.get("scores", {}),
        "final_review": my_result.get("final_review", "")
    }
    
    my_output_path = os.path.join(output_dir, "review_my_paper.json")
    with open(my_output_path, 'w', encoding='utf-8') as f:
        json.dump(my_review_result, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 결과 저장: {my_output_path}")
else:
    print("⚠️ 위 셀에서 논문을 입력해주세요.")

---
## 정리

### 이번 노트북에서 배운 것

1. **Agent 개념**: LLM + 워크플로우 = Agent
2. **LangGraph**: Node와 Edge로 워크플로우 구성
3. **단계별 처리**: 각 단계의 결과를 확인하며 진행
4. **7차원 평가**: Stanford Agentic Reviewer 방식

### 다음 노트북 예고

**Notebook 4: Few-shot Learning으로 리뷰 품질 향상**
- Nature Communications의 실제 리뷰 예시 사용
- 기본 Agent vs Few-shot Agent 비교
- 어떤 리뷰가 더 유용한지 판단